In [ ]:
from request_post import *
import pandas as pd
import pygsheets
import threading
import time
import random

In [ ]:
import configparser
config = configparser.ConfigParser()
config.read('./config.ini')
email = config['fb_account']['email']
passw = config['fb_account']['passw']


In [ ]:
auth_file = "./aqueous-flames-396208-687ba5889c5d.json"
gc = pygsheets.authorize(service_file = auth_file)
sheet_url = "https://docs.google.com/spreadsheets/d/1YfT73wC3EhEUt7vNMRN4gPNRbKlcSAs18gFfTq9hhGs/edit#gid=0" 

In [ ]:
sheet = gc.open_by_url(sheet_url)
sheet_test01 = sheet.worksheet_by_title("三千則文章")

In [ ]:
pre_urls = []
for i in sheet_test01.get_all_records():
    pre_urls.append(i['url'])

In [ ]:
all_url = list(set(pre_urls+pd.read_csv("url_3000.csv")['0'].unique().tolist()))

In [ ]:
all_url_tot = []
for i in all_url:
    if 'https://mbasic.facebook.com/groups/1786425331505061/permalink' in i:
        all_url_tot.append(i)

In [ ]:
sheet_test01.update_value('A1', "url")
sheet_test01.update_value('B1', "user")
sheet_test01.update_value('C1', "text")
sheet_test01.update_value('D1', "img links")
sheet_test01.update_value('E1', "blog title")
sheet_test01.update_value('F1', "blog link")
sheet_test01.update_value('G1', "comment")
sheet_test01.update_value('H1', "datetime")
sheet_test01.update_value('I1', "likes")
sheet_test01.update_value('J1', "memo")


In [ ]:
# 設定新的

main_driver = setDriver(email,passw,"https://mbasic.facebook.com/groups/1786425331505061")
page_driver = setDriver(email,passw)

In [ ]:
# 還沒爬的文章
url_list = pd.read_csv("url_3000.csv")['0'].unique().tolist()

In [ ]:

result_list = []
# assert(len(url_list) == count_pre_urls)
def requestsMain(main_driver,url_list):
    url_count = len(all_url_tot)
    while url_count<=3000:
        links = getLink(main_driver,email,passw)
        if len(links)>0:
            for l in links:
                sl = getShortUrl(l)
                if sl in all_url_tot:
                    pass
                else:
                    url_list.append(sl)
                    url_count+=1
            # url_list = list(set(url_list))
        else:
            continue
        
        click_flag = False
        while click_flag==False:
            click_flag=clickNext(main_driver,email,passw)
        print("click the new pages")
        if url_count%10 == 0:
            saveUnprocessUrl(url_list)
            
def requestsContent(page_driver,url_list,result_list,need_result=3000,cur_point=1):
    
    while len(result_list)<=need_result-len(result_list):
        if len(url_list)>0:
            url = url_list.pop(0)
            res = getContent(page_driver,url,email,passw)
            # print(url)
            # print(res)
            result_list.append(res)
            saveDataToSheet(url,res,len(result_list)+cur_point)
            
            
            if len(result_list)%50 == 0:
                saveDataToCsv(result_list)
                saveUnprocessUrl(url_list)
            time.sleep(random.uniform(1,3))
            
    print("finish crawlering")
        
def saveDataToCsv(result_list):
    pd.DataFrame(result_list).to_csv('result_3000.csv',index=False)
def saveDataToSheet(url,data:dict,current_point):
    sheet_test01.update_value('A'+str(current_point), url)
    sheet_test01.update_value('B'+str(current_point), data['user_name'])
    sheet_test01.update_value('C'+str(current_point), data['text'])
    sheet_test01.update_value('D'+str(current_point), str(data['imgs']))
    sheet_test01.update_value('E'+str(current_point), data['blog_title'])
    sheet_test01.update_value('F'+str(current_point), data['blog_url'])
    sheet_test01.update_value('G'+str(current_point), str(data['comment']))
    sheet_test01.update_value('H'+str(current_point), data['timestamp'])
    sheet_test01.update_value('I'+str(current_point), data['likes'])
    sheet_test01.update_value('J'+str(current_point), data['memo'])
    
def saveUnprocessUrl(url_list):
    pd.DataFrame(url_list).to_csv('url_3000.csv',index=False)

In [ ]:
t1 = threading.Thread(target=requestsMain,args=(main_driver,url_list,))
t2 = threading.Thread(target=requestsContent,args=(page_driver,url_list,result_list))



In [ ]:
t1.start()
t2.start()